# ASSIGNMENT: Cartoon Face Mask
## This assignment tests your ability to apply face recognition, image masks, image thresholding, video capturing and saving video feed into a video file concepts.

## TASK: Cartoonify faces in video feed from live webcam

### Steps
- 1. **Capture video** feed from webcam
- 2. **Recognize faces** in the video
- 3. **Replace/Mask the face** region with your favorite cartoon character
- 4. **Save the video** feed into a video file
- 5. Submit the following files
    - Notebook (.ipynb)
    - Video (.avi/.mp4)

# Import Libraries and Setup Paths

In [4]:
import cv2
import os
import numpy as np

# Set up paths for Haar Cascade XML files
cascPathface = "/Users/bishakha/Downloads/haarcascade_frontalface_alt2.xml"
cascPatheyes = "/Users/bishakha/Downloads/haarcascade_eye_tree_eyeglasses.xml"

# Load Haar Cascade classifiers
faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)


# Load Cartoon Mask

In [6]:
# Load the cartoon mask (ensure it's a PNG with transparency)
mask = cv2.imread('cute.png', cv2.IMREAD_UNCHANGED)  # Load with alpha channel


# Initialize Webcam Video Capture

In [8]:
# Initialize webcam video capture
video_capture = cv2.VideoCapture(0)


# Set Up Video Writer to Save Processed Video

In [10]:
# Set up video writer to save the processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out = cv2.VideoWriter('masked_video.mp4v', fourcc, 20.0, (640, 480))  # Output video file


# Process Video Frames and Detect Faces

In [12]:
# Process video frames and detect faces
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    # Loop over detected faces
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Resize the cartoon mask to match the face's width and height
        resized_mask = cv2.resize(mask, (w, h))

        # Check if the mask has an alpha channel (transparency)
        if resized_mask.shape[2] == 4:  # Has an alpha channel (PNG format)
            alpha_mask = resized_mask[:, :, 3] / 255.0  # Normalize alpha values (0 to 1)
            mask_rgb = resized_mask[:, :, :3]  # Extract the RGB part of the mask (without alpha)
        else:
            alpha_mask = np.ones((h, w))  # No transparency, so full opacity
            mask_rgb = resized_mask  # Use the entire mask without alpha blending
        
        # Extract the face region of interest (ROI) from the frame
        faceROI = frame[y:y+h, x:x+w]

        # Apply the mask using alpha blending
        for c in range(3):  # For R, G, B channels
            faceROI[:, :, c] = (1 - alpha_mask) * faceROI[:, :, c] + alpha_mask * mask_rgb[:, :, c]

        # Put the blended face region back into the frame
        frame[y:y+h, x:x+w] = faceROI

        # Detect eyes and draw circles around them (as per original code)
        eyes = eyeCascade.detectMultiScale(faceROI)
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)

    # Display the resulting frame with cartoon mask overlay
    cv2.imshow('Face Video with Mask', frame)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


2025-03-27 03:08:42.717 python[31406:2631312] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-27 03:08:42.717 python[31406:2631312] +[IMKInputSession subclass]: chose IMKInputSession_Modern


# Release Resources and Close Windows

In [14]:
# Release the video capture and writer objects
video_capture.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()
